In [1]:
import pandas as pd
from PIL import Image
import numpy as np
import os

# Cargar el CSV
csv_path = 'angle.csv'
df = pd.read_csv(csv_path)

print(df.columns)
# Función para cargar una imagen y convertirla a un array numpy
def load_image(image_path):
    with Image.open('image_bank/' + image_path) as img:
        img_array = np.array(img)
        #img_array = img_array / 255.0  # Normalizar los valores de los píxeles
        return img_array

# Cargar todas las imágenes
df['image_bank'] = df['image_path'].apply(load_image)

# Mostrar el dataframe resultante
print(df.head())

# Separar las características y las etiquetas
X = np.stack(df['image_data'].values)
y = df[' steering_angle'].values

print(f'X shape: {X.shape}, y shape: {y.shape}')


Index(['2024-06-2118-02-29.png', '0.0'], dtype='object')


KeyError: 'image_path'

In [2]:
import imgaug.augmenters as iaa

# Definir una secuencia de aumentaciones
augmentation = iaa.Sequential([
    iaa.Fliplr(0.5),  # Flip horizontal
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},  # Escalar
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},  # Trasladar
        rotate=(-10, 10),  # Rotar
        shear=(-8, 8)  # Sesgar
    ),
    iaa.GaussianBlur(sigma=(0, 1.0)),  # Desenfoque gaussiano
    iaa.Multiply((0.8, 1.2)),  # Cambiar el brillo
    iaa.ContrastNormalization((0.8, 1.2))  # Cambiar el contraste
])

# Aplicar las aumentaciones a las imágenes
#augmented_images = augmentation(images=X)

# Verificar las dimensiones de las imágenes aumentadas
#print(f'Augmented images shape: {augmented_images.shape}')


/usr/local/lib/python3.11/dist-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `ContrastNormalization()` is deprecated. Use `imgaug.contrast.LinearContrast` instead.
  warn_deprecated(msg, stacklevel=3)


In [3]:
augmented_images = X / 255.0
augmented_images = augmented_images.astype(np.float32)

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split

# Barajar los datos antes de la división
indices = np.arange(len(augmented_images))
np.random.shuffle(indices)

X = augmented_images[indices]
Y = y[indices]

# Dividir el dataset en entrenamiento y validación (80-20)
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

# Verificar las dimensiones de los conjuntos de entrenamiento y validación
print(f'Training set: X_train shape: {X_train.shape}, y_train shape: {y_train.shape}')
print(f'Validation set: X_val shape: {X_val.shape}, y_val shape: {y_val.shape}')


Training set: X_train shape: (48928, 64, 128), y_train shape: (48928,)
Validation set: X_val shape: (12232, 64, 128), y_val shape: (12232,)


In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Definir el modelo CNN simplificado
model = Sequential([
    Conv2D(8, (3, 3), activation='relu', input_shape=(64, 128, 1)),  # Ajustar para un canal de entrada
    MaxPooling2D((2, 2)),
    
    Conv2D(16, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    Dense(64, activation='relu'),  # Reducir el tamaño de la capa densa
    Dropout(0.5),
    Dense(1)  # Salida para predicción del ángulo
])

# Compilar el modelo
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Resumen del modelo
model.summary()



2024-06-16 03:46:21.158135: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
2024-06-16 03:46:22.861177: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-16 03:46:22.880681: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 126, 8)     │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 63, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 61, 16)     │         1,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 30, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 28, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 14, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2688)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       172,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 178,049 (695.50 KB)

 Trainable params: 178,049 (695.50 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# Entrenar el modelo
history = model.fit(
    X_train, y_train,
    epochs=40,
    batch_size=32,
    validation_data=(X_val, y_val)
)

# Evaluar el modelo en el conjunto de validación
val_loss, val_mae = model.evaluate(X_val, y_val)
print(f'Validation Loss: {val_loss}, Validation MAE: {val_mae}')


2024-06-16 03:46:24.075057: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1603272704 exceeds 10% of free system memory.
2024-06-16 03:46:24.974174: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1603272704 exceeds 10% of free system memory.


Epoch 1/40


I0000 00:00:1718509586.133150   16297 service.cc:145] XLA service 0x7fb038005000 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1718509586.133188   16297 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-06-16 03:46:26.163510: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-16 03:46:26.263895: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8906


 103/1529 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0028 - mae: 0.0338

I0000 00:00:1718509587.868888   16297 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1529/1529 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.0012 - mae: 0.0200 - val_loss: 5.1012e-04 - val_mae: 0.0143
Epoch 2/40
1529/1529 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 5.0314e-04 - mae: 0.0146 - val_loss: 3.2637e-04 - val_mae: 0.0117
Epoch 3/40
1529/1529 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 4.8528e-04 - mae: 0.0140 - val_loss: 2.8137e-04 - val_mae: 0.0108
Epoch 4/40
1529/1529 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 4.4738e-04 - mae: 0.0134 - val_loss: 3.3435e-04 - val_mae: 0.0113
Epoch 5/40
1529/1529 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 3.8842e-04 - mae: 0.0129 - val_loss: 2.2564e-04 - val_mae: 0.0098
Epoch 6/40
1529/1529 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 3.6318e-04 - mae: 0.0124 - val_loss: 2.2090e-04 - val_mae: 0.0095
Epoch 7/40
1529/1529 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 3.3512e-04 - mae: 0.0120 - val_loss: 2.4003e-04 - val_mae: 0.0095
Epoch 8/40
1529/1529 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 3.3593e-04 - mae: 0.0119 - val_loss: 2.0263e-04 - val_mae

In [7]:
model.save('steering_angle_model_town.keras')